This notebook will be for data cleaning

In [26]:
import pandas as pd
import pickle 
import matplotlib.pyplot as plot 
%matplotlib inline

In [27]:
mta = pd.read_csv('https://data.ny.gov/api/views/ypri-5iwa/rows.csv?accessType=DOWNLOAD&sorting=true')

In [ ]:
with open('my_dataframe.pickle', 'wb') as to_write:
    pickle.dump(mta, to_write)
mta.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
16233622,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2021 12:00:00 AM,04:00:00,REGULAR,5554,541
16233623,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2021 12:00:00 AM,08:00:00,REGULAR,5554,541
16233624,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2021 12:00:00 AM,12:00:00,REGULAR,5554,541
16233625,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2021 12:00:00 AM,16:00:00,REGULAR,5554,541
16233626,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2021 12:00:00 AM,20:00:00,REGULAR,5554,541


In [42]:
with open('my_dataframe.pickle','rb') as read_file:
    new_pickled_data = pickle.load(read_file)
#new_noise.head()    
#removing unnecessary columns
#mta.drop(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME','DIVISION','DESC'],
#        inplace=True,
#        axis=1)
#new_pickled_data.head

In [43]:
new_pickled_data['DATE'] = pd.to_datetime(new_pickled_data['DATE'])

In [44]:
new_pickled_data.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
16233622,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-01-01,04:00:00,REGULAR,5554,541
16233623,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-01-01,08:00:00,REGULAR,5554,541
16233624,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-01-01,12:00:00,REGULAR,5554,541
16233625,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-01-01,16:00:00,REGULAR,5554,541
16233626,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,2021-01-01,20:00:00,REGULAR,5554,541
